In [1]:
from flask import Flask, render_template, request
import nltk
from nltk.stem import PorterStemmer
from nltk.corpus import stopwords
from nltk.sentiment.vader import SentimentIntensityAnalyzer 
import re
import pickle
import warnings
warnings.filterwarnings("ignore")

In [2]:
stemmer = PorterStemmer()
sia = SentimentIntensityAnalyzer()
vec = pickle.load(open("vectorizer.pkl","rb"))
selector = pickle.load(open("selector.pkl","rb"))
model = pickle.load(open("voting_Model.pkl","rb"))

In [3]:
def textProcessing_Vectorizing(text):
    text = text.lower()
    text = re.sub("^[a-z]"," ",text)
    text = [stemmer.stem(word) for word in nltk.word_tokenize(text) if word not in stopwords.words("english")]
    text = " ".join(text)
    vec_text = vec.transform([text]).toarray()
    vec_text = selector.transform(vec_text)
    return vec_text
def polarityScores(text):
    sentimentScores = sia.polarity_scores(text)
    if sentimentScores["compound"]<0:
        return "negative"
    elif sentimentScores["compound"]==0:
        return "neutral"
    elif sentimentScores["compound"]>0:
        return "positive"

In [4]:
app = Flask(__name__)

@app.route('/')
def home():
    return render_template('index.html')

@app.route('/predict', methods=['POST'])
def predict():
    text = request.form['text']
    vec_text = textProcessing_Vectorizing(text)
    model_prediction = model.predict(vec_text)[0]
    vader_prediction = polarityScores(text)
    if vader_prediction=="positive":
        definition="A text that expresses a favorable or optimistic view towards a subject or event.\
        It may convey happiness, satisfaction, excitement, admiration, or other positive emotions."
    elif vader_prediction=="negative":
        definition="A text that expresses an unfavorable or pessimistic view towards a subject or event.\
        It may convey sadness, anger, disappointment, frustration, or other negative emotions."
    elif vader_prediction=="neutral":
        definition="A text that expresses a factual or objective view towards a subject or event without \
        showing any emotional bias. It may provide information or describe something without conveying any \
        particular feeling or judgment."
    return render_template('index.html',head=vader_prediction.upper(),prediction1=model_prediction,prediction2=vader_prediction,definition=definition)

if __name__ == '__main__':
    app.run(debug=False)

 * Serving Flask app '__main__' (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)
127.0.0.1 - - [09/Jun/2023 16:15:19] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [09/Jun/2023 16:15:19] "GET /favicon.ico HTTP/1.1" 404 -
127.0.0.1 - - [09/Jun/2023 16:21:30] "POST /predict HTTP/1.1" 200 -
127.0.0.1 - - [09/Jun/2023 16:21:36] "POST /predict HTTP/1.1" 200 -
127.0.0.1 - - [09/Jun/2023 16:21:41] "POST /predict HTTP/1.1" 200 -
127.0.0.1 - - [09/Jun/2023 16:22:16] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [09/Jun/2023 16:22:31] "POST /predict HTTP/1.1" 200 -
127.0.0.1 - - [09/Jun/2023 16:22:36] "POST /predict HTTP/1.1" 200 -
127.0.0.1 - - [09/Jun/2023 16:22:42] "POST /predict HTTP/1.1" 200 -
